In [ ]:
cd ../..

In [ ]:
# Hot reload packages
%load_ext autoreload
%autoreload 2

In [ ]:
# Show image in notebook
%matplotlib inline

In [ ]:
# Show log output in Notebook
import logging
import sys
log_level = logging.INFO
root = logging.getLogger()
root.setLevel(log_level)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(log_level)
root.addHandler(handler)

# Load model

In [ ]:
from src.persistence.persistence import load_model_and_pre_processing

MODEL_DIR = 'output/model'

[model, image_encoder, label_encoder] = load_model_and_pre_processing(MODEL_DIR)

# Load data

In [ ]:
from datasets.load_data import load_raw
limit = 100
skip=[]  # ids to skip
img_dim = [250, 250]
img_dir = f'datasets/dataset_3b_ZO_AnB_other_production/images/{img_dim[0]}x{img_dim[1]}/'
label_dir = 'datasets/dataset_3b_ZO_AnB_other_production/labels/'
[Img, Data, Label, _] = load_raw(img_dir, label_dir, skip, limit)

# Pre process data

In [ ]:
print('Img.shape: ', Img.shape)
X = image_encoder.transform(Img)
print('X.shape: ', X.shape)

In [ ]:
print('Label.shape: ', Label.shape)
Z = label_encoder.transform(Label).toarray()
print('Z.shape: ', Z.shape)

# Predict

In [ ]:
predictions = model.predict(X, verbose=1)

# Show results

In [ ]:
from src.notebook.image_display import show_image

# Code to show specific image by index
index = 0
id = Data[index, 2]
print(id)
image = Img[index]
show_image(image)

In [ ]:
from src.stats import show_prediction_list
show_prediction_list(predictions, Z)

In [ ]:
from src.notebook.image_display import show_prediction_images
print("prediction images:")
show_prediction_images(
    Img,
    Z,
    predictions,
    Data[:, 2],
    label_encoder,
    50
)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, fixed

from src.notebook.results import combined_report
    
widget = widgets.FloatSlider(
    value=0.9,
    min=0.5,
    max=1.0,
    step=0.005,
    continuous_update=False,
    description='Threshold:',
    readout=True,
    readout_format='.3f',
)

interact(
    combined_report,
    threshold=widget,
    true_oh=fixed(Z),
    predictions_oh=fixed(predictions),
    ids=fixed(Data[:, 2]),
    label_encoder=fixed(label_encoder)
);
